In [21]:
# export
import torch
import torch.nn as nn
from fastai.vision.all import *
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
from fastscript import *
import pandas as pd
from pathlib import Path
import time
import warnings
from datetime import datetime

In [22]:
#export 
from modules_th.video_block import *
from modules_th.inflator import *
from modules_th.triplet_loss import *
from modules_th.supcon_module import *
from modules_th.cus_cbs import *
from modules_th.ucf_cnn import *

In [23]:
#export
@call_parse
def main(gpu    :Param("GPU to run on", int)=None,
         file   :Param("csv path", bool)='UCF_experiments/train_UCF.csv',
         n_lbl  :Param("# of different labels per batch", int)=4,
         n_el   :Param("# of elements per label", int)=2,
         l      :Param("Target number of frames of the ResizeTime transform", int)=60,
         skip   :Param("skip frames",int)=20,
         size     :Param("size for Resize", int)=224,
         n_epoch:Param("# of epochs to train", int)=10,
         loss   :Param("Choose a loss between CEL-SCL and SCL ", str)='CEL-SCL',
         embs_size:Param("embeddings size", int)=128,
         n_views:Param("number of views", int)=2,
         log_nm :Param("logs file name. if 'date' the name will be composed by the date", str)='date',
         descr  :Param("description of the experiment", str)='First training with mixed SupConLoss to have a preformance baseline; first trial with a large skip'
        ):
    prefix = '/mnt/data/eugeniomarinelli/'
    
    if gpu is not None:
        gpu = setup_distrib(gpu)
        items = rank0_first(read_data)
    else:
        items = read_data()

    learn = get_learner(items, loss, l, size, n_lbl , n_el, skip, embs_size,n_views)
    print('Learner Loaded \n')
    
    
    # set up logs file
    now = datetime.now()
    time = now.strftime("%d_%m_h%H")
    logs_file = prefix+'logs_'+time+'.csv' if log_nm == 'date' else prefix+'logs_'+log_nm+'.csv'
    Logs_csv =   CSVLogger(fname= logs_file, append=False) 

    torch.cuda.empty_cache()
    if gpu is not None:
        print("Distributed Data Parallel training started")
        with learn.distrib_ctx(gpu):
            learn.fine_tune(n_epoch,cbs=[Logs_csv])
    else:
        warnings.filterwarnings("ignore", message='.*nonzero.*', category=UserWarning)
        print("Data Parallel training started")
        with learn.parallel_ctx(device_ids=[0,1]):
            learn.fine_tune(n_epoch,cbs=[Logs_csv])

    learn.save(prefix+'UCF_experiments/trained_models_cnn/ucf_cnn_'+time)
    save_CEL_SCL_losses(learn, logs_file)
    experiment = pd.DataFrame({'date':time,
             'description':descr,
             'l':l,
             'skip':skip,
             'size':size,
             'loss':loss,
             'n_el':n_el,
             'n_lbl':n_lbl,
             'n_epochs':n_epoch}).to_csv(prefix+'UCF_experiments/experiments_logs.csv', mode='a')
    

In [24]:
now = datetime.now()
time = now.strftime("%d_%m_h%H")

In [25]:
items = read_data()
learn = get_learner(items.iloc[:100], loss='CEL-SCL', l=1, size=64, n_lbl=4, n_el=4, skip=2, embs_size=128, n_views=2)
print('Learner Loaded \n')

prefix = '/mnt/data/eugeniomarinelli/'
# set up logs file
logs_file = prefix+'logs_'+time+'.csv'
Logs_csv =   CSVLogger(fname=logs_file, append=False) 

Learner Loaded 



In [26]:
learn.fine_tune(1, cbs=[Logs_csv])

TypeError: iteration over a 0-d tensor

In [ ]:
# default_exp distributed_training

In [29]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ucf.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted 07_read_video_function.ipynb.
Converted 08_experiments_on_images.ipynb.
Converted 09_create_embeddings.ipynb.
Converted CustomCallbacks.ipynb.
Converted SupConModule.ipynb.
Converted Transformers.ipynb.
Converted UCF_SupCon.ipynb.
Converted distributed_training.ipynb.
Converted index.ipynb.
